In [1]:
import pandas as pd
import numpy as np
from pymoo.optimize import minimize
import pickle

from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
from pymoo.visualization.scatter import Scatter
import contact
from notebooks.optimization_problems.constraints import Requirements
from pyreport import PlotUtil
import matplotlib.pyplot as plt

%matplotlib

plt.close('all')

Using matplotlib backend: Qt5Agg


In [2]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "telesat_post_processed.h5"

row_limit = -1

store_500km = pd.HDFStore(file_path)

instances_500km_df = store_500km.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_500km_df = instances_500km_df.sort_index(0)

In [3]:
# problem, setting, res = pickle.load(open('Telesat_nsga3_200_5000.pkl', 'rb'))
# problem, setting, res = pickle.load(open('Telesat_rnsga2_100_25.pkl', 'rb'))
# problem, setting, res = pickle.load(open('Telesat_unsga3_200_50.pkl', 'rb'))
# problem, setting, res = pickle.load(open('Telesat_unsga3_500_125.pkl', 'rb'))
problem, setting, res = pickle.load(open('Telesat_unsga3_100_25.pkl', 'rb'))
problem, setting, res = pickle.load(open('Telesat_unsga3_400_100.pkl', 'rb'))
problem, setting, res = pickle.load(open('Telesat_downselect_unsga3_600_150.pkl', 'rb'))

print("n_gen: %d" % res.algorithm.n_gen)
pass

n_gen: 5100


In [4]:
x_pass = res.X[:, problem.x_indices['pass']].astype('bool')
for i in range(x_pass.shape[0]):
    x_pass[i, :] = contact.down_select_passes(x_pass[i, :], problem.O_matrix)

x_Ptx_dBm = res.X[:, problem.x_indices['power']].astype('float64')
x_Ptx_dBm[~x_pass] = np.NaN
x_Ptx_dBm = np.nanmax(x_Ptx_dBm, axis=1)
#x_modcod = np.squeeze(res.X[:, problem.x_indices['modcod']].astype('int64'))

# Recompute all four objectives
f_pointing = np.empty_like(res.F[:,0])
f_energy = np.empty_like(res.F[:,0])
f_latency = np.empty_like(res.F[:,0])
f_throughput = np.empty_like(res.F[:,0])

scale_factors = np.array([1/-1e9, 1, 1/1e3, 1])

for i in range(len(f_throughput)):
    ff, gg = problem.evaluate_unmasked(res.X[i,:])
    ff = ff * scale_factors
    f_pointing[i] = ff[3] # s
    f_energy[i] = ff[2] # Kilo Joule
    f_latency[i] = ff[1]   # s
    f_throughput[i] = ff[0]   # Gigabit

pass

KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
points = []
F_norm = res.F / np.max(res.F, axis=0)

# Max throughput point
ind = np.argmax(f_throughput)
p = np.array([f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind]])
points.append({'ind': ind, 'p': p, 'color': 'r', 'marker': '.', 'size': 10, 'label': 'Max T\n(T=%d Gbit, L=%d s, E=%d kJ, P=%d s)' % (f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind])})

# # Max E, minimum P point
# ref = np.array([1, 0])
# ind = np.argmin(np.linalg.norm(F_norm[:,2:4] - ref, axis = 1))
# p = np.array([f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind]])
# points.append({'ind': ind, 'p': p, 'color': 'g', 'marker': '.', 'size': 10, 'label': 'Max E, Min P\n(T=%d Gbit, L=%d s, E=%d kJ, P=%d s)' % (f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind])})
#
# # 1000 GBit, min Energy
# ref = np.array([-1000e9, 0])
# ind = np.argmin(np.linalg.norm((res.F[:,[0,2]] - ref) / np.max(res.F[:,[0,2]] - ref, axis = 0), axis = 1))
# p = np.array([f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind]])
# points.append({'ind': ind, 'p': p, 'color': 'b', 'marker': '.', 'size': 10, 'label': 'T = 1000 GBit, Min E\n(T=%d Gbit, L=%d s, E=%d kJ, P=%d s)' % (f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind])})

# 3600 s, min Energy
# ref = np.array([3600, 0])
# ind = np.argmin(np.linalg.norm((res.F[:,[1,2]] - ref) / np.max(res.F[:,[0,2]] - ref, axis = 0), axis = 1))
# p = np.array([f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind]])
# points.append({'ind': ind, 'p': p, 'color': 'y', 'marker': '.', 'size': 10, 'label': 'L = 3600 s, Min E\n(T=%d Gbit, L=%d s, E=%d kJ, P=%d s)' % (f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind])})

# 3600 s, min Energy
# ref = np.array([3600, 0])
# ind = np.argmin(np.linalg.norm((res.F[:,[1,2]] - ref) / np.max(res.F[:,[0,2]] - ref, axis = 0), axis = 1))
# p = np.array([f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind]])
# points.append({'ind': ind, 'p': p, 'color': 'y', 'marker': '.', 'size': 10, 'label': 'L = 3600 s, Min E\n(T=%d Gbit, L=%d s, E=%d kJ, P=%d s)' % (f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind])})

# # Reference points
# ref_points = setting['algorithm'].survival.ref_points / np.array([-1e9, 1e3, 1])
# points.append({'ind': ind, 'p': ref_points[0,[1,2,0]], 'color': 'r', 'marker': 'x', 'size': 15, 'label': 'Target 1\n(%d Gbit, %d kJ, %d s)' % (ref_points[0,0], ref_points[0,1], ref_points[0,2])})
# points.append({'ind': ind, 'p': ref_points[1,[1,2,0]], 'color': 'g', 'marker': 'x', 'size': 15, 'label': 'Target 2\n(%d Gbit, %d kJ, %d s)' % (ref_points[1,0], ref_points[1,1], ref_points[1,2])})

In [ ]:
T_lim = (0, 4000)
L_lim = (0, 2000)
E_lim = (0, 150)
P_lim = (0, 10000)

limits_on = False

# offset 0 limit with 10%
perc = 0.025
T_lim = np.array(T_lim) - np.array([perc * (T_lim[1] - T_lim[0]), 0])
L_lim = np.array(L_lim) - np.array([perc * (L_lim[1] - L_lim[0]), 0])
E_lim = np.array(E_lim) - np.array([perc * (E_lim[1] - E_lim[0]), 0])
P_lim = np.array(P_lim) - np.array([perc * (P_lim[1] - P_lim[0]), 0])

In [ ]:
fig = plt.figure(figsize=(3.2*3, 2.4*3))
axs = fig.subplots(3,3, gridspec_kw={'height_ratios': [1, 1, 1]})
axs = axs.flatten()

ax = axs[0]
ax.grid(True)
ax.scatter(f_energy, f_throughput, marker='.', s=1)
for point in points:
    ax.scatter(point['p'][2], point['p'][0],
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

ax.set_xlabel("Energy used [kJ]")
ax.set_ylabel("Throughput [Gbit]")
if limits_on:
    ax.set_xlim(E_lim)
    ax.set_ylim(T_lim)
ax.set_axisbelow(True)

ax = axs[1]
ax.grid(True)
ax.scatter(f_pointing, f_throughput, marker='.', s=1)
for point in points:
    ax.scatter(point['p'][3], point['p'][0],
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

ax.set_xlabel("Pointing [s]")
ax.set_ylabel("Throughput [Gbit]")
if limits_on:
    ax.set_xlim(P_lim)
    ax.set_ylim(T_lim)
ax.set_axisbelow(True)

ax = axs[2]
ax.grid(True)
ax.scatter(f_latency, f_throughput, marker='.', s=1)
for point in points:
    ax.scatter(point['p'][1], point['p'][0],
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

# ax.scatter(p_maxTminP[0], p_maxTminP[1], color='b', s=10)
# ax.scatter(p_minEminP[0], p_minEminP[1], color='y', s=10)
ax.set_xlabel("Avg. Latency [s]")
ax.set_ylabel("Throughput [Gbit]")
if limits_on:
    ax.set_xlim(L_lim)
    ax.set_ylim(T_lim)
ax.set_axisbelow(True)

ax = axs[3]
ax.grid(True)
ax.scatter(f_energy, f_latency, marker='.', s=1)
for point in points:
    ax.scatter(point['p'][2], point['p'][1],
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

ax.set_xlabel("Energy used [kJ]")
ax.set_ylabel("Avg. Latency [s]")
if limits_on:
    ax.set_xlim(E_lim)
    ax.set_ylim(L_lim)
ax.set_axisbelow(True)

ax = axs[4]
ax.grid(True)
ax.scatter(f_pointing, f_latency, marker='.', s=1)
for point in points:
    ax.scatter(point['p'][3], point['p'][1],
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

ax.set_xlabel("Pointing [s]")
ax.set_ylabel("Avg. Latency [s]")
if limits_on:
    ax.set_xlim(P_lim)
    ax.set_ylim(L_lim)
ax.set_axisbelow(True)

ax = axs[5]
ax.grid(True)
ax.scatter(f_energy, f_pointing, marker='.', s=1)
for point in points:
    ax.scatter(point['p'][2], point['p'][3],
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

# ax.scatter(p_maxTminP[0], p_maxTminP[1], color='b', s=10)
# ax.scatter(p_minEminP[0], p_minEminP[1], color='y', s=10)
ax.set_xlabel("Energy used [kJ]")
ax.set_ylabel("Pointing [s]")
if limits_on:
    ax.set_xlim(E_lim)
    ax.set_ylim(P_lim)
ax.set_axisbelow(True)

axs[6].set_axis_off()
axs[7].set_axis_off()
ax = axs[8]
for point in points:
    ax.scatter(np.nan, np.nan,
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

ax.legend(fontsize=8)
ax.set_axis_off()

plt.tight_layout()

# autoAxis = sub1.axis()
# rec = Rectangle((autoAxis[0]-0.7,autoAxis[2]-0.2),(autoAxis[1]-autoAxis[0])+1,(autoAxis[3]-autoAxis[2])+0.4,fill=False,lw=2)
# rec = sub1.add_patch(rec)
# rec.set_clip_on(False)

#fig.savefig('D:/git/thesis_report_ae/figures/full_pareto_six_%s.pdf' % setting['label'])

In [ ]:
ind = points[0]['ind']
pass_ind = np.nonzero(x_pass[ind,:])[0]

ff, gg = problem.evaluate_unmasked(res.X[ind,:])
pass

In [ ]:
T_orbit = 5680
T_sim = T_orbit * 3

fig = plt.figure(figsize=(3.2, 2.4))

# Plot all passes
for i, pass_df in instances_500km_df.groupby(level=0):
    p = pass_df.index[0][0] - 1 # Pass index
    tof = pass_df.tof.values
    d = pass_df.d.values/1000
    line, = plt.plot(tof, d, linewidth=0.1, color='tab:grey')
    #plt.text(tof[0], d[0], '%d' % p)

# Plot used passes
for i, pass_df in instances_500km_df.groupby(level=0):
    p = pass_df.index[0][0] - 1 # Pass index
    tof = pass_df.tof
    d = pass_df.d/1000
    if p in pass_ind:
        line, = plt.plot(tof, d, linewidth=0.5, color='tab:red')


plt.xlabel('Time of flight [s]')
plt.ylabel('Range [km]')
plt.xlim((0, T_sim))

plt.grid()
PlotUtil.apply_report_formatting()
fig.set_size_inches(3.2*2, 2.4, forward=True)

plt.tight_layout()
plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/telesat_range_selection.pdf')
plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/telesat_range_selection_%s_%d.svg' % (setting['label'], res.algorithm.n_gen))
